In [ ]:
from io_processing.marpower import read_amcs_excel
import io_processing.sql.pika as io_pika
from pypika import Query, Column

%load_ext autoreload
%autoreload 2

In [ ]:
df = read_amcs_excel("../io_lists/52422003_3210_AMCS IO-List R1.11.xlsx")
df

In [ ]:
root_topic = "amcs/+"


def mqtt_source(topic):
    mqtt_connector = io_pika.MqttConnector(
        "mqtt://vernemq:1883",
        topic,
        "at_least_once",
        "power-hub",
        "power-hub",
        "bytes",
    )

    table = Query.create_table("amcs").columns(
        io_pika.proctime_col(), io_pika.data_col()
    )

    return io_pika.table_with_connector(
        table, mqtt_connector, [io_pika.Include("partition", "topic")]
    )


print(str(mqtt_source(root_topic)))

In [ ]:
from io_processing.marpower import normalize_amcs_io_list
import polars
import polars as pl
import yaml

from io_processing.components import Components
from io_processing.io_list import IoFlavor
from io_processing.sql.dbt import sql_for_io

io_list = normalize_amcs_io_list(df)
components = Components("../components.yaml")
orphans = components.orphan_signals(io_list)

a = io_list.select(
    "yard_tag", polars.struct(polars.exclude("yard_tag")).alias("signals")
)
grouped = (
    io_list.select(
        "yard_tag", polars.struct(polars.exclude("yard_tag")).alias("signals")
    ).group_by("yard_tag").all()
)

combined = components.combine_io(io_list, IoFlavor.AMCS).filter(pl.col("type") == "Pump")

a, b,c= sql_for_io(components._description, combined, IoFlavor.AMCS)
# a = (components.combine_io(io_list, IoFlavor.AMCS).with_columns(pl.col("signals").list.eval(pl.element().struct.field("value_name")).alias("value_names"))
#         .select(
#             pl.col("system"),
#             pl.col("type"),
#             pl.struct(pl.exclude("system", "type")).alias("components"),
#             pl.col("value_names")
#         )
#         .group_by([pl.col("system"), pl.col("type")])
#         .agg(pl.col("components"), pl.col("value_names").flatten()))
# missing = components.missing_components(io_list)

# io_list.join(components._components, ("yard_tag", "tag"), "left")
# a = components.combine_io(io_list).filter(pl.col("signals").list.len() > 1).with_columns(pl.col("signals").list.eval(pl.element().struct.field("Tag")).alias("tags")).select(pl.exclude("signals"))
# grouped
# io_list["type"].unique()

In [ ]:
print(a)
print(b)

In [ ]:
print(c["bilge_pump"][0])

In [ ]:
print(c["bilge_pump"][1])

In [ ]:
from io_processing.data_gen.gen import Generator

genny = Generator(combined, components._description, IoFlavor.AMCS)

In [ ]:
import polars as pl

vitters_io = polars.read_excel("io_lists/hydraulic-plc/9428050-4 Sail-Hydraulic PLC IO list.xlsx", sheet_name="IO list", read_options={"header_row": 24})
vitters_cabling = polars.read_excel("io_lists/hydraulic-plc/9428051-4 Sail PLC cable list.xlsx", read_options={"header_row": 10})
with_components = vitters_io.with_columns(
  polars.col("PLC var-Name").str.extract(r"^([a-zA-Z_\d]+)\..*").alias("component")
)
joined_with_cabling = (
    with_components.filter((pl.col("component").is_not_null()))
    .join(vitters_cabling, left_on="1st cable no.", right_on="Cable no.", how="left")
    .select(
        pl.col("component"),
        pl.col("1st cable no."),
        pl.col("Source pos").alias("first source pos"),
        pl.col("Target pos").alias("first target"),
        pl.col("PLC var-Name"),
        pl.col("2nd cable no."),
    )
    .join(vitters_cabling, left_on="2nd cable no.", right_on="Cable no.", how="left")
    .select(
        pl.col("component"),
        pl.col("1st cable no."),
        pl.col("first source pos"),
        pl.col("first target"),
        pl.col("Source pos").alias("second source poc"),
        pl.col("Target pos").alias("second target"),
        pl.col("PLC var-Name"),
        pl.col("2nd cable no."),
    )
)

In [ ]:

first_cable_join = vitters_cabling.join(
  vitters_io,
  left_on="Cable no.",
  right_on="1st cable no."
)
second_cable_join = vitters_cabling.join(
  vitters_io,
  left_on="Cable no.",
  right_on="2nd cable no."
)

missing = (
    set(vitters_io["PLC var-Name"].to_list())
    - set(first_cable_join["PLC var-Name"].to_list())
    - set(second_cable_join["PLC var-Name"].to_list())
)

blabla = vitters_io.filter(polars.col("PLC var-Name").is_in(missing) & polars.col("IO box / Location").is_in(["KEEL", "MST"]))


# (polars.col("Cable no.") == polars.col("2nd cable no."))